In [1]:
import numpy as np 
import tensorflow as tf 
from collections import Counter

import utils.load_data as load_data
from models.emotion_recognition import ClassifyEmotion

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data, orig_lengths, labels, metadata = load_data.load_sentences(max_sequence_length=750, 
                                                                win_len=.04, win_step=.02)

Finished data session Session1
Finished data session Session2
Finished data session Session3
Finished data session Session4
Finished data session Session5


In [4]:
# Convert Y to one hot
int_labels = LabelEncoder().fit_transform(labels)
int_labels = int_labels.reshape(len(int_labels), 1)
onehot_labels = OneHotEncoder().fit_transform(int_labels).toarray()

In [12]:
# Subset limited amount of data for hyperparameter search 
lim_data = data[::5]
lim_lengths = orig_lengths[::5] 
lim_onehot_labels = onehot_labels[::5]

# Split into train and test
lim_train_x, lim_val_x, lim_train_y, lim_val_y, lim_train_lengths, lim_val_lengths = train_test_split(lim_data,
                                                                              lim_onehot_labels,
                                                                              lim_lengths,
                                                                              test_size=0.1)

# Hyperparameter search using limited data 

In [22]:
learning_rates = [.0001, .0005, .0007, .001]
truth = np.argmax(lim_val_y, 1) 

for lr in learning_rates:
    lstm = ClassifyEmotion(lr=lr, num_hidden=256)
    print(vars(lstm))
    val_pred = lstm.train(train_x=lim_train_x, val_x=lim_val_x, 
                          train_y=lim_train_y, val_y=lim_val_y, 
                          train_lengths=lim_train_lengths, 
                          val_lengths=lim_val_lengths, 
                          verbose=False, 
                          batch_size=32, epochs=50, load_model=False, 
                          save_path='final_model/lr_test.ckpt')
    acc = sum(val_pred == np.argmax(lim_val_y, 1))/len(val_pred)
    print('Accuracy: {}'.format(acc))

{'num_features': 26, 'num_classes': 6, 'learning_rate': 0.0001, 'num_hidden': 256}
Created 42 batches of size 32
Created Validation 5 batches of size 32
Data Batch Shape: (32, 750, 26) Labels Shape: (32, 6) 
(?, 512)
(?, 512)
(?, 128)
(?, 6)
Epoch 0 mean batch val loss 1.7027305364608765
Epoch 5 mean batch val loss 1.6789894104003906
Epoch 10 mean batch val loss 1.6803194284439087
Epoch 15 mean batch val loss 1.6795856952667236
Epoch 20 mean batch val loss 1.679100751876831
Epoch 25 mean batch val loss 1.678067922592163
Epoch 30 mean batch val loss 1.6766560077667236
Epoch 35 mean batch val loss 1.6761207580566406
Epoch 40 mean batch val loss 1.6761928796768188
Epoch 45 mean batch val loss 1.6723095178604126
Accuracy: 0.19594594594594594
{'num_features': 26, 'num_classes': 6, 'learning_rate': 0.0005, 'num_hidden': 256}
Created 42 batches of size 32
Created Validation 5 batches of size 32
Data Batch Shape: (32, 750, 26) Labels Shape: (32, 6) 
(?, 512)
(?, 512)
(?, 128)
(?, 6)
Epoch 0 me

In [25]:
num_hidden = [512, 128, 64]

for h in num_hidden: 
    lstm = ClassifyEmotion(lr=.0005, num_hidden=h)
    print(vars(lstm))
    val_pred = lstm.train(train_x=lim_train_x, val_x=lim_val_x, 
                          train_y=lim_train_y, val_y=lim_val_y, 
                          train_lengths=lim_train_lengths, 
                          val_lengths=lim_val_lengths, 
                          verbose=False, 
                          batch_size=32, epochs=50, load_model=False, 
                          save_path='final_model/num_hidden_test.ckpt')
    acc = sum(val_pred == np.argmax(lim_val_y, 1))/len(val_pred)
    print('Accuracy: {}'.format(acc))

{'num_features': 26, 'num_classes': 6, 'learning_rate': 0.0005, 'num_hidden': 512}
Created 42 batches of size 32
Created Validation 5 batches of size 32
Data Batch Shape: (32, 750, 26) Labels Shape: (32, 6) 
(?, 1024)
(?, 1024)
(?, 256)
(?, 6)
Epoch 0 mean batch val loss 1.6801049709320068
Epoch 5 mean batch val loss 1.6791181564331055
Epoch 10 mean batch val loss 1.6758193969726562
Epoch 15 mean batch val loss 1.6723060607910156
Epoch 20 mean batch val loss 1.7721226215362549
Epoch 25 mean batch val loss 1.736720323562622
Epoch 30 mean batch val loss 1.733214020729065
Epoch 35 mean batch val loss 1.728166937828064
Epoch 40 mean batch val loss 1.7222931385040283
Epoch 45 mean batch val loss 1.715497612953186
Accuracy: 0.25
{'num_features': 26, 'num_classes': 6, 'learning_rate': 0.0005, 'num_hidden': 128}
Created 42 batches of size 32
Created Validation 5 batches of size 32
Data Batch Shape: (32, 750, 26) Labels Shape: (32, 6) 
(?, 256)
(?, 256)
(?, 64)
(?, 6)
Epoch 0 mean batch val los

In [27]:
# Dense layer hidden units = 512 
lstm = ClassifyEmotion(lr=.0005, num_hidden=64, dense_hidden=512)
print(vars(lstm))
val_pred = lstm.train(train_x=lim_train_x, val_x=lim_val_x, 
                      train_y=lim_train_y, val_y=lim_val_y, 
                      train_lengths=lim_train_lengths, 
                      val_lengths=lim_val_lengths, 
                      verbose=False, 
                      batch_size=32, epochs=50, load_model=False, 
                      save_path='final_model/num_hidden_test.ckpt')

{'num_features': 26, 'num_classes': 6, 'learning_rate': 0.0005, 'num_hidden': 64, 'dense_hidden': 512}
Created 42 batches of size 32
Created Validation 5 batches of size 32
Data Batch Shape: (32, 750, 26) Labels Shape: (32, 6) 
(?, 128)
(?, 128)
(?, 512)
(?, 6)
Epoch 0 mean batch val loss 1.6846727132797241
Epoch 5 mean batch val loss 1.6806259155273438
Epoch 10 mean batch val loss 1.6762855052947998
Epoch 15 mean batch val loss 1.6878869533538818
Epoch 20 mean batch val loss 1.6715717315673828
Epoch 25 mean batch val loss 1.666272521018982
Epoch 30 mean batch val loss 1.6656242609024048
Epoch 35 mean batch val loss 1.6539623737335205
Epoch 40 mean batch val loss 1.6508935689926147
Epoch 45 mean batch val loss 1.6635900735855103
Epoch 49 mean batch val loss 1.6635900735855103


In [28]:
acc = sum(val_pred == np.argmax(lim_val_y, 1))/len(val_pred)
print('Accuracy: {}'.format(acc))

Accuracy: 0.20270270270270271


# Train using all data 

In [5]:
# Split into train and test
train_x, val_x, train_y, val_y, train_lengths, val_lengths = train_test_split(data,
                                                                              onehot_labels,
                                                                              orig_lengths,
                                                                              test_size=0.1)

In [ ]:
lstm = ClassifyEmotion(lr=.0005, num_hidden=128, dense_hidden=512)
print(vars(lstm))
val_predictions = lstm.train(train_x=train_x, val_x=val_x, train_y=train_y, val_y=val_y, 
                             train_lengths=train_lengths, val_lengths=val_lengths, 
                             batch_size=32, epochs=20, load_model=False, verbose=True,
                             save_path='final_model/emotion_recognition_750.ckpt')

{'num_hidden': 128, 'num_classes': 6, 'num_features': 26, 'learning_rate': 0.0005, 'dense_hidden': 512}
Created 208 batches of size 32
Created Validation 24 batches of size 32
Data Batch Shape: (32, 750, 26) Labels Shape: (32, 6) 
(?, 256)
(?, 256)
(?, 512)
(?, 6)
Epoch 0 mean loss 1.7397600412368774
Epoch 0 mean batch val loss 2.1266140937805176
Epoch 1 mean loss 1.7312729358673096
Epoch 2 mean loss 1.7293425798416138
Epoch 3 mean loss 1.7284976243972778
Epoch 4 mean loss 1.7279680967330933
